## Scrapping urbania

The goal of this notebook is to scrape data from https://urbania.pe/ and store it on a mongo databse

### Libraries import

In [220]:
import math
import numpy as np
import pymongo
import re 
import urllib.request as urllib2
import datetime

from time import sleep
from bs4 import BeautifulSoup

from constants import *

import pandas as pd

In [2]:
# bsae urls to use
base_url = 'https://urbania.pe{}'
apartments_url = 'https://urbania.pe/buscar/venta-de-departamentos'


### Define database connections

In [5]:
uri = "mongodb://{}:{}@{}/{}".format(username, password, hostlist, database)
client = pymongo.MongoClient(uri)
db = client['urbania']

In [29]:
list(db['admin'].find())

[{'_id': ObjectId('5deb3aba3dadd940d2148a36'), 'name': 'gonzalo'},
 {'_id': ObjectId('5dec891299df02824e747081'), 'name': 'jay'}]

In [7]:
apartmentsdb = db['apartments']

### Here we put the logic to get the data

In [13]:
def getSoupObj(url):
    sleep(1.5)
    try:
        r = urllib2.urlopen(url)
    except UnicodeEncodeError as e:
        return None
    html = r.read()
    htmltext = html.decode('ISO-8859-2')
    return BeautifulSoup(htmltext, 'lxml')

In [159]:
dictionary = {
    'Tipos Dptos.': 'apartment_type',
    'Dpto.': 'department',
    'Provincia': 'province',
    'Distrito': 'district',
    'Urb.': 'urbanization',
    'Torres': 'towers',
    'Inmuebles': 'properties',
    'Ascensores': 'elevators',
    'NÂ°pisos': 'n_floors',
    'Cocheras': 'garages',
    'Referencia': 'reference',
    'Construye': 'builds',
    'Comercializa': 'sells',
    'Financia': 'finances'
}

In [309]:
tihs = 10000
count = 1
apartments = []
fails = 0
threshold = 100000000
counter = 0
# count is for page counter
while count < 5:
    hdr = {'User-Agent': 'Mozilla/5.0'}
    print(apartments_url)
    req = urllib2.Request(apartments_url, headers=hdr)
    apartmentsBsObj = getSoupObj(req)
    print(type(apartmentsBsObj))
#     objs = apartmentsBsObj.findAll('article', {'class': 'b-card-item js-card-item b-card-item--proyecto-destacado'})
     
    objs = apartmentsBsObj.findAll('div', {'class': 'posting-card highlighted'})
#     objsSeen = apartmentsBsObj
    print('In page {} we found {} apartments'.format(count, len(objs)))
    for obj in objs:
#         the objA is the object with <a> label in obj
        counter +=1
        apartment = {}
        apartment['url'] = obj['data-to-posting']
        apartment['_id'] = int(apartment['url'].split('-')[-1])
        if apartment['_id'] < threshold:
            try:
                get_apartment(apartment)
            except:
                
                threshold = apartment['_id'] if (apartment['_id'] < threshold) else threshold
                fails+=1
                print('damn so')
        else:
            print('xd')
        print(apartment)
        apartments.append(apartment)
        
#         if counter == 10:
#             break
        break

        
    count +=1
    break

pd.DataFrame(apartments)

https://urbania.pe/buscar/venta-de-departamentos
<class 'bs4.BeautifulSoup'>
In page 1 we found 14 apartments
{'url': '/inmueble/proyecto-altea-condominio-lima-magdalena-inmobiliaria-eureka-sac-empresa-8003929', '_id': 8003929, 'title': 'Altea Condominio', 'address': 'Jr. Hermilio Valdizan NÂ°150 Magdalena, Magdalena, Lima', 'description': '\n\nALTEA es un exclusivo proyecto situado en una ubicaciĂłn privilegiada frente al mar. A pasos del malecĂłn Grau de Magdalena de Mar, donde podrĂĄs relajarte, practicar deportes y actividades familiares con la complicidad de la naturaleza. Cuenta con departamentos Flats y DĂşplex desde 52 hasta 106 M2 de 2 y 3 dormitorios. La calidad se refleja en el diseĂąo, distribuciĂłn de espacios, ventilaciĂłn y luminosidad en todos sus ambientes, asĂ\xad como tambiĂŠn en los acabados de primer nivel. Respecto a las ĂĄreas comunes, contamos con sala de niĂąos en el primer piso, la cual se vincula con un patio central con jardĂ\xadn vertical. Asimismo, en el p

,Dormitorios,Entrega,Estacionamientos,Metros techados,Metros totales,Unidades,_id,address,description,status,title,url
0,\n2 a 3\n,\n-\n,\n - \n,\n50 a 105 mÂ˛\n,\n50 a 105 mÂ˛\n,\n54\n,8003929,"Jr. Hermilio Valdizan NÂ°150 Magdalena, Magdal...",\n\nALTEA es un exclusivo proyecto situado en ...,\n\t\t\t\t\tPre-venta en construcciĂłn\n\t\t\t\t,Altea Condominio,/inmueble/proyecto-altea-condominio-lima-magda...


In [291]:
def get_apartment(apartment):
    
    apartment_url = base_url.format(apartment['url'])
#     print(apartment_url)
    req = urllib2.Request(apartment_url)
    apartmentObj = getSoupObj(req)
#     print(apartmentObj)
    
    if apartmentObj == -1:
        return
#     print(apartment['url'])
    get_text(apartment, apartmentObj, 'title', 'info-title', 'h2')
#     print(apartment)
    get_text(apartment, apartmentObj, 'address', 'info-location', 'h2')
    get_text(apartment, apartmentObj, 'description', 'section-description', 'div')
#     
#     get_text(apartment, apartmentObj, 'status', 'status done current', 'div', 0)
    new_feature = apartmentObj.find('div', {'class': 'status done current'})
    new_feature = new_feature.find('span', {'class': 'label'})
    apartment['status'] = new_feature.text
    
    get_status(apartment, apartmentObj)


In [295]:
def get_status(obj, objBs):
#     status-columns
    status_columns = objBs.find('div', {'class': 'status-columns'})
    columns=status_columns.findAll('div', {'class': 'column'})
#     get_columns_info(obj, columns)
#     pd.Series(columns).apply(get_columns_info,(obj))
#     print(len(columns))
    pd.Series(columns).apply(get_columns_info, args= (obj,))

In [306]:
def get_columns_info(column, obj):
#     print('hello')
#     print(column)
    rows=column.findAll('div', {'class': 'row'})
    pd.Series(rows).apply(get_rows_info, args=(obj,))

In [308]:
def get_rows_info(row, obj):
#     print(row)

    label = row.find('span', {'class': 'label'})
    data = row.find('span', {'class': 'data'})
#     print(label, row)
    obj[label.text] = data.text

In [245]:
class building:
    def __init__(self, buildingType):
        self.buildingType = buildingType
    
    

In [27]:
class apartment:
    def __init__(self):
        

SyntaxError: invalid syntax (<ipython-input-27-b7843a038499>, line 2)

In [197]:
def get_text(obj, objBs, feature, tag, marker, inplace=True):
    new_feature = objBs.find(marker, {'class': tag})
#     print(new_feature)
    if inplace:
        obj[feature] = new_feature.text
    else:
        return new_feature.text

In [179]:
def get_general_info(general_info, obj, option= 1):
    general_info = general_info[1:]
    general_info = general_info[:-1]
    
    if option:
        state_delivery = general_info[0].split('\n')
        obj['status'] = state_delivery[0]

        if len(state_delivery)>1:
            obj['delivery_date'] = state_delivery[1]
    index = option

    for i in range(index, len(general_info)):
    
        data_aux = general_info[i].split('\n')
        data_aux = remove_blanks(data_aux, '')
#         index_data = len(data_aux)
        obj[dictionary[data_aux[0]]]= data_aux[1]
        

In [178]:
def remove_blanks(current_list, pattern):
    final_list = [item for item in current_list if elem != pattern]
    return final_list

In [ ]:
def get_real_estate(real_estate):
    